# Capstone Project: IBM Datascience
## Data, Explarotory Analysis and Machine Learning Section

## Solomin Oleg

## May 2020

## DATA SECTION

### Libraries

In [1]:
import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

from IPython.display import display

print('Libraries imported.')

Libraries imported.


## Manhattan neighborhoods

Manhattan area is quite big so it would be apporpriate to limit area by Lower Manhattan and Midtown.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
NYC_neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
NYC_neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
for data in NYC_neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NYC_neighborhoods = NYC_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
addressNYC = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
locationNYC = geolocator.geocode(addressNYC)
latitudeNYC = locationNYC.latitude
longitudeNYC = locationNYC.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitudeNYC, longitudeNYC))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [7]:
manhattan_data = NYC_neighborhoods[NYC_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Manhattan is quite long enough so it's appropriate to limit area by low part of it.

In [8]:
manhattan_data = manhattan_data[manhattan_data['Latitude'] < 40.77].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Chinatown,40.715618,-73.994279
1,Manhattan,Lenox Hill,40.768113,-73.958860
2,Manhattan,Roosevelt Island,40.762160,-73.949168
3,Manhattan,Clinton,40.759101,-73.996119
4,Manhattan,Midtown,40.754691,-73.981669


In [9]:
addressM = 'Financial District, NY'

geolocator = Nominatim(user_agent="ny_explorer")
locationM = geolocator.geocode(addressM)
latitudeM = locationM.latitude
longitudeM = locationM.longitude
print('The geographical coordinate of Financial District NYC are {}, {}.'.format(latitudeM, longitudeM))

The geographical coordinate of Financial District NYC are 40.7076124, -74.009378.


In [10]:
# create map of Manhattan using latitude and longitude values with minor corrections in order to better locate area
map_manhattan = folium.Map(location=[latitudeM+0.025, longitudeM+0.03], zoom_start=13)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  

display(map_manhattan)

## Downtown Toronto neighborhoods

In [11]:
# Url to get Data from Wikipedia concerning "List of postal codes of Canada: M"
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [12]:
raw_data=pd.read_html(url)

In [13]:
df=raw_data[0]

In [14]:
df_adj=df.dropna(axis=0)

In [15]:
url_coord = 'http://cocl.us/Geospatial_data'

coord=pd.read_csv(url_coord)

#coord_adj = coord.rename(columns={'Postal Code' : 'Postal code'})

In [16]:
T_neighborhoods=pd.merge(df_adj,coord, on='Postal Code')

In [17]:
T_neighborhoods.drop(columns=['Postal Code'], inplace = True)

In [18]:
downtown_data = T_neighborhoods[T_neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [19]:
addressT = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
locationT = geolocator.geocode(addressT)
latitudeT = locationT.latitude
longitudeT = locationT.longitude
print('The geographical coordinate of Downtown Toronto are {}, {}.'.format(latitudeT, longitudeT))

The geographical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [20]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitudeT, longitudeT], zoom_start=13)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
display(map_downtown)

## Explore neighboorhoods in Manhattan and Downtown Toronto

Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = '===' # your Foursquare ID
CLIENT_SECRET = '===' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ok!')
print('CLIENT_SECRET: ok!')

Your credentails:
CLIENT_ID: ok!
CLIENT_SECRET: ok!


In [22]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Chinatown
Lenox Hill
Roosevelt Island
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Gramercy
Battery Park City
Financial District
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [24]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [25]:
print(manhattan_venues.shape)
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))
manhattan_venues.head()

(2137, 7)
There are 294 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown,40.715618,-73.994279,Cheeky Sandwiches,40.715821,-73.991830,Sandwich Place
1,Chinatown,40.715618,-73.994279,Kiki's,40.714476,-73.992036,Greek Restaurant
2,Chinatown,40.715618,-73.994279,Hotel 50 Bowery NYC,40.715936,-73.996789,Hotel
3,Chinatown,40.715618,-73.994279,Renew Day Spa,40.715559,-73.996747,Spa
4,Chinatown,40.715618,-73.994279,Michaeli Bakery,40.714704,-73.991847,Bakery


In [26]:
barberM = manhattan_venues[manhattan_venues['Venue Category'].str.contains("Barber")]
print('There are {} barbershops.'.format(len(barberM['Venue Category'])))

There are 19 barbershops.


In [27]:
print(downtown_venues.shape)
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))
downtown_venues.head()

(1205, 7)
There are 204 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [28]:
barberT = downtown_venues[downtown_venues['Venue Category'].str.contains("Barber")]
print('There are {} barbershops.'.format(len(barberT['Venue Category'])))

There are 6 barbershops.


## Barbershops in Manhattan

Manhattan neighborhoods even in a lower part is twice crowded with venues. So there'll be a tough competition but a better chance to make a fortune. Let's take a closer look at barbershop's location.

In [29]:
# create map of Manhattan Barbershops using latitude and longitude values
map_manhattan_venue = folium.Map(location=[latitudeM+0.025, longitudeM+0.03], zoom_start=13)

# add markers to map
for lat, lng, label in zip(barberM['Venue Latitude'], barberM['Venue Longitude'], barberM['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_venue)  

display(map_manhattan_venue)

Now we can make a segmentation say in three area using simple but efficient k-means method.

The KMeans class has many parameters that can be used, but we will use these three:

* **init**: Initialization method of the centroids.
Value will be: 'k-means++'. k-means++ selects initial cluster centers for k-means clustering in a smart way to speed up convergence.

* **n_clusters**: The number of clusters to form as well as the number of centroids to generate.
Value will be: 3 (say we have 3 centers)

* **n_init**: Number of times the k-means algorithm will be run with different centroid seeds. The final results will be the best output of n_init consecutive runs in terms of inertia.
Value will be: 12

Initialize KMeans with these parameters, where the output parameter is called k_means.

In [30]:
k_means = KMeans(init="k-means++", n_clusters=3, n_init=12)

In [31]:
X=barberM.values[:,4:6]

In [32]:
k_means.fit(X)
labels = k_means.labels_

In [33]:
barberM["Labels"] = labels

/Users/olegsolomin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


We need to define a small function to color venues according to segmentation.

In [34]:
def color(cluster): 
    if cluster == 0: 
        col = 'green'
    elif cluster == 1: 
        col = 'blue'
    else: 
        col='red'
    return col 

Now we are ready to show Manhattan's barbershops on a map with clusterization.

In [35]:
# create map of Manhattan Barbershops with Clusters using latitude and longitude values
map_manhattan_venue_cluster = folium.Map(location=[latitudeM+0.025, longitudeM+0.03], zoom_start=13)

# add markers to map
for lat, lng, label,cluster in zip(barberM['Venue Latitude'], barberM['Venue Longitude'], barberM['Venue'], barberM['Labels']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color(cluster),
        fill=True,
        fill_color=color(cluster),
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan_venue_cluster)  

display(map_manhattan_venue_cluster)

Fine. We can locate three areas with targeted venues. But for better understanding it's appropriate to add more venues "for men". For example we add following keywords: barber, gym, steak, sport, health, cycle.

In [36]:
mensM = manhattan_venues[manhattan_venues['Venue Category'].str.contains("barber|gym|steak|sport|health|cycle",case=False)]
print('There are {} mens places.'.format(len(mensM['Venue Category'])))

There are 167 mens places.


Much better! We have tenfold places to cluster.

In [37]:
XM=mensM.values[:,4:6]

In [38]:
k_means.fit(XM)
labelsM = k_means.labels_

In [39]:
mensM["Labels"] = labelsM

/Users/olegsolomin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


We need to define another small function to color barbershop in different way.

In [40]:
def fill(barber): 
    if barber == 'Salon / Barbershop': 
        fill = True
    else: 
        fill= False
    return fill 

In [41]:
# create map of Manhattan Mens Places with Clusters using latitude and longitude values
map_manhattan_mens_cluster = folium.Map(location=[latitudeM+0.025, longitudeM+0.03], zoom_start=13)

# add markers to map
for lat, lng, label,cluster, barber in zip(mensM['Venue Latitude'], mensM['Venue Longitude'], mensM['Venue'], mensM['Labels'], mensM['Venue Category']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color=color(cluster),
        fill=fill(barber),
        fill_color=color(cluster),
        fill_opacity=1,
        parse_html=False).add_to(map_manhattan_mens_cluster)  

display(map_manhattan_mens_cluster)

# Conclusions

Venues clusterazation helps to define areas and better understand environment.